In [ ]:
""""
Plots the total value of assets (BTC and EUR) over time in a given GDAX account, taking into account the fluctuating value of BTC.

"""

In [ ]:
# EDIT: enter the number of days trading history to review
trading_history_days = 1

In [ ]:
!pip install gdax
!pip install python-dateutil==2.6.1

In [ ]:
import gdax
import numpy
import pandas
import json
import pprint
import matplotlib as plt
import dateutil
import datetime
import pytz

%matplotlib inline

now = datetime.datetime.utcnow()
n_days_ago = now - datetime.timedelta(days=trading_history_days)

# value gives the total value of our btc and euro assets, at a given time.
# df2[value][index] is the value at the time given by the row numbered index.

btc_stack_cache = {}
eur_stack_cache = {}

# fill the btc_stack column
def get_btc_stack(index, dataframe):
    index = int(index)
    
    if index in btc_stack_cache:
        return btc_stack_cache[index]
    
    if index == -1:
        current_size = starting_btc_stack
    else:
        side = dataframe['side'][index]
        current_transaction_btc = float(dataframe['size'][index])
        # last_size = dataframe['btc_stack'][index-1]
        last_size = get_btc_stack(index = index-1, dataframe = dataframe)
        if side == "buy":
            current_size = last_size + current_transaction_btc
        if side == "sell":
            current_size = last_size - current_transaction_btc
    
    btc_stack_cache[index] = current_size
    return current_size

# fill the eur_stack column
def get_eur_stack(index, dataframe):
    index = int(index)
    
    if index in eur_stack_cache:
        return eur_stack_cache[index]
    
    if index == -1:
        current_size = starting_eur_stack
    else:
        side = dataframe['side'][index]
        current_transaction_btc = float(dataframe['size'][index])
        current_transaction_price = float(dataframe['price'][index])        
        # last_size = dataframe['eur_stack'][index-1]
        last_size = get_eur_stack(index = index-1, dataframe = dataframe)
        if side == "buy":
            current_size = float(last_size - current_transaction_btc * current_transaction_price)
        if side == "sell":
            current_size = float(last_size + current_transaction_btc * current_transaction_price)
            
    eur_stack_cache[index] = current_size
    return current_size          

def asset_value(index, dataframe):
    index = int(index)
    if index == -1:
        current_value = starting_btc_stack * price + starting_eur_stack
    else:
        current_price = float(dataframe['price'][index])
        current_eur = float(dataframe['eur_stack'][index])
        current_btc = float(dataframe['btc_stack'][index])
        current_value = current_btc * current_price + current_eur
    return current_value

def asset_value_btc(index, dataframe):
    index = int(index)
    if index == -1:
        current_value = starting_btc_stack + starting_eur_stack / price
    else:
        current_price = float(dataframe['price'][index])
        current_eur = float(dataframe['eur_stack'][index])
        current_btc = float(dataframe['btc_stack'][index])
        current_value = current_btc + current_eur / current_price
    return current_value

def asset_value_btc_v2(index, dataframe):
    return dataframe['value'][index] / dataframe['price'][index]

In [ ]:
b64secret, key, passphrase = [x.split('"')[1] for x in open("production/env.sh").readlines()]
auth_client = gdax.AuthenticatedClient(key, b64secret, passphrase)
public_client = gdax.PublicClient()

In [ ]:
accts = auth_client.get_accounts()
for acct in accts:
    if acct["currency"] == "EUR":
        eur_acct_id = acct["id"]
    if acct["currency"] == "BTC":
        btc_acct_id = acct["id"]

In [ ]:
accts

In [ ]:
eur_acct_history = auth_client.get_account_history(eur_acct_id)
btc_acct_history = auth_client.get_account_history(btc_acct_id)

In [ ]:
all_eur_acct_history = []

# TODO: use the list of transfers in/out of the account to label our graph, and 
# differentiate between changes in asset value due to incoming/outgoing cash, versus
# gains/losses due to trading
eur_acct_transfers = []
btc_acct_transfers = []

for page in eur_acct_history:
    for item in page:
        all_eur_acct_history.append(item)
        if item["type"] == "transfer":
            eur_acct_transfers.append((item["amount"], item["created_at"]))
print len(all_eur_acct_history)
print eur_acct_transfers

all_btc_acct_history = []
for page in btc_acct_history:
    for item in page:
        all_btc_acct_history.append(item)
        if item["type"] == "transfer":
            btc_acct_transfers.append((item["amount"], item["created_at"]))
print len(all_btc_acct_history)
print btc_acct_transfers


In [ ]:
#price = 0.0

# get initial balances for eur and btc accounts
# this now needs to be the balance at the point in time set by the trading_history_days value

starting_eur_stack = 0.0
starting_btc_stack = 0.0

# iterate over account history from oldest to newest
for item in reversed(all_eur_acct_history):
    created_at = dateutil.parser.parse(item["created_at"])
    created_at = created_at.replace(tzinfo=None)
    if created_at > n_days_ago:
        starting_eur_stack = float(item["balance"])
        break

for item in reversed(all_btc_acct_history):
    created_at = dateutil.parser.parse(item["created_at"])
    created_at = created_at.replace(tzinfo=None)
    if created_at > n_days_ago:
        starting_btc_stack = float(item["balance"])
        break

# NB this doesn't work because we added more bitcoin and more euros :(

# request is list of pages, page is list of dicts. each dict is a fill. page lists fills in reverse chronological order.
request = auth_client.get_fills()

In [ ]:
all_results = []
for page in request:
    for row in page:
        created_at = dateutil.parser.parse(row["created_at"])
        created_at = created_at.replace(tzinfo=None)
        if created_at > n_days_ago:
            all_results.append(row)
df = pandas.DataFrame(data=list(reversed(all_results)))

In [ ]:
#make a new dataframe using columns from the initial dataframe
df2 = df[["created_at", "order_id", "price", "size", "side"]]

# let initial btc price be the price it has for the first recorded fill in account history
price = float(df2["price"][0])

In [ ]:
# Get the times in timestamp format so trades are plotted at sensible intervals (ignore the warning)
df2['created_at'] = pandas.to_datetime(df2['created_at'])

In [ ]:
btc_stack = pandas.Series()
eur_stack = pandas.Series()
value = pandas.Series()
value_btc = pandas.Series()
value_holding = pandas.Series()
difference = pandas.Series()
df3 = pandas.concat([df2, btc_stack, eur_stack, value, value_holding, difference], axis=1, ignore_index=False)
df3.columns=["created_at", "order_id", "price", "size", "side", "btc_stack", "eur_stack", "value", "value_holding", "difference"]

In [ ]:
for i in range(0, len(all_results)):
    df3.at[i, 'btc_stack'] = get_btc_stack(i, df3)
    df3.at[i, 'eur_stack'] = get_eur_stack(i, df3)
    df3.at[i, 'value'] = asset_value(i, df3)
    df3.at[i, 'value_holding'] = starting_btc_stack * float(df3['price'][i]) + starting_eur_stack
    df3.at[i, 'difference'] = df3.at[i, 'value'] - df3.at[i, 'value_holding']

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(24, 12))
labels = []
ax = df3.plot(ax=ax, kind='line', x='created_at', y='value', c="green")
ax = df3.plot(ax=ax, kind='line', x='created_at', y='value_holding', c="red")
lines, _ = ax.get_legend_handles_labels()
ax.legend(lines, ["value trading", "value holding"], loc='best')
plt.show()

In the plot, the green line shows the value of the total assets held, in BTC and EUR, over time, given the trades that have taken place in the account.

The red line shows, counterfactually, what the total value of the assets _would have been_ had you not traded, but instead held your starting BTC, given the fluctuating currency prices.

The values are given in EUR.

In [ ]:
import matplotlib.dates as mdates
plt.rcParams.update({'font.size': 20})

#df3.plot(x="created_at", y="difference", title="profit over holding", figsize=(24, 12))

fig, ax = plt.subplots(figsize=(24, 12))
labels = []
ax = df3.plot(ax=ax, kind='line', x='created_at', y='difference', c="green", title="profit over holding")
xfmt = mdates.DateFormatter('%d-%m-%y %H:%M')
ax.xaxis.set_major_formatter(xfmt)
lines, _ = ax.get_legend_handles_labels()
ax.legend(lines, ["value trading", "value holding"], loc='best')
plt.show()

In [ ]:
df4 = df3.tail(100)
df4.plot(x = "created_at", y = "value", title = "asset value in euro over time", figsize = (12, 12))

In [ ]:
#TODO:
# fix floating point rounding
# df2.append rows as they come in, reload graph every time